In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from src.augmentations import CustomAugmentations

from copy import deepcopy
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score
from tqdm.notebook import tqdm
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D, Conv2D, Dropout, BatchNormalization

import numpy as np
import random
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)


In [7]:
height,width=32,32
batch_size=64
num_classes=10

In [8]:
from src.kerasCNN import *

In [9]:
net_configs = [
{"kernel_sizes":[3],
"num_filters":[32], 
"fc_sizes":[128],},

{"kernel_sizes":[3, 3],
"num_filters":[32, 64], 
"fc_sizes":[128, 64],},


{"kernel_sizes":[3, 3, 3],
"num_filters":[32, 64, 64], 
"fc_sizes":[128, 64, 64],},

{"kernel_sizes":[5],
"num_filters":[32], 
"fc_sizes":[128],},

{"kernel_sizes":[5, 5],
"num_filters":[32, 64], 
"fc_sizes":[128, 64],},
]



In [10]:
for config in net_configs:
    config_str = '_'.join([f'{k}_{v}' for k,v in config.items()])
    histories = list()
    accuracies = list()
    class_accuracies = list()
    conf_mats = list()

    for i in tqdm(range(5), desc='Iteration'):
        train_set = tf.keras.preprocessing.image_dataset_from_directory(
        '../data/CINIC10/train',
        labels='inferred',
        label_mode='int',
        class_names=None,
        color_mode='rgb',
        batch_size=64,
        image_size=(height, width),
        shuffle=True,
        seed=i,
        validation_split=0.05,
        subset='training'
        )

        val_set = tf.keras.preprocessing.image_dataset_from_directory(
            '../data/CINIC10/valid',
            labels='inferred',
            label_mode='int',
            class_names=None,
            color_mode='rgb',
            batch_size=64,
            image_size=(height, width),
            shuffle=True,
            seed=i,
            validation_split=0.05,
            subset='validation'
        )

        test_set = tf.keras.preprocessing.image_dataset_from_directory(
            '../data/CINIC10/test',
            labels='inferred',
            label_mode='int',
            class_names=None,
            color_mode='rgb',
            batch_size=64,
            image_size=(height, width),
            shuffle=False,
        )

        dnn_model = create_cnn(**config)

        print(f'Fitting model {i}')
        history = dnn_model.fit(train_set, validation_data=val_set, epochs=40)

        print(f'Predicting model {i}')
        preds = dnn_model.predict(test_set, verbose=2)
        preds = preds.argmax(axis=1)
        classes = test_set.class_names
        test_labels = list()
        for images, labels in test_set:
            class_labels = [int(label) for label in labels]
            test_labels.extend(class_labels)
        test_labels = np.array(test_labels)

        conf_mat = confusion_matrix(test_labels, preds)
        accuracy = accuracy_score(test_labels, preds)
        class_accuracy = conf_mat.diagonal()/conf_mat.sum(axis=1)

        print(f'Saving model {i}')
        dnn_model.save(f'../models/ClassicCNN_{config_str}_no_aug_iter{i}.h5')
        histories.append(deepcopy(history.history))
        accuracies.append(accuracy)
        class_accuracies.append(class_accuracy)
        conf_mats.append(conf_mat)

    
    # save histories
    import pickle
    with open(f'results/HISTORY_ClassicCNN_{config_str}_no_aug.pkl', 'wb') as f:
        pickle.dump(histories, f)

    # save accuracies
    with open(f'results/ACCURACY_ClassicCNN_{config_str}_no_aug.pkl', 'wb') as f:
        pickle.dump(accuracies, f)

    # save class accuracies
    with open(f'results/CLASS_ACCURACY_ClassicCNN_{config_str}_no_aug.pkl', 'wb') as f:
        pickle.dump(class_accuracies, f)

    # save confusion matrices
    with open(f'results/CONF_MAT_ClassicCNN_{config_str}_no_aug.pkl', 'wb') as f:
        pickle.dump(conf_mats, f)




Iteration:   0%|          | 0/5 [00:00<?, ?it/s]

Found 90000 files belonging to 10 classes.
Using 85500 files for training.
Found 90000 files belonging to 10 classes.
Using 4500 files for validation.
Found 90000 files belonging to 10 classes.
Fitting model 0
Epoch 1/40
1336/1336 [==============================] - 16s 12ms/step - loss: 1.6215 - accuracy: 0.4107 - val_loss: 1.4910 - val_accuracy: 0.4518
Epoch 2/40
1336/1336 [==============================] - 15s 11ms/step - loss: 1.3646 - accuracy: 0.5015 - val_loss: 1.4516 - val_accuracy: 0.4853
Epoch 3/40
1336/1336 [==============================] - 16s 12ms/step - loss: 1.2478 - accuracy: 0.5469 - val_loss: 1.5168 - val_accuracy: 0.4642
Epoch 4/40
1336/1336 [==============================] - 16s 12ms/step - loss: 1.1524 - accuracy: 0.5807 - val_loss: 1.4928 - val_accuracy: 0.4791
Epoch 5/40
 125/1336 [=>............................] - ETA: 14s - loss: 1.1128 - accuracy: 0.5960